<a href="https://colab.research.google.com/github/mathewpolonsky/Marketplace-Item-Matching/blob/main/Making_Submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1ID_cljhCgJZ6Ht7VJPqIkH0Jx5KVJbOH
!gdown 10K-1goB8dM4TeJwU98qb_M03TfXTUDfm
!gdown 1WnuHe62ECL36ZZdRbsmKrAO5kkhmLdKz

!unzip rubert-base-class-full.zip
!unzip regression_model.zip
!unzip distilbert-quora-full.zip

In [ ]:
!gdown 1-7aNio3oHeZAg_t1dvTePLMKEvHZ8TmT
!gdown 1N8fUo17cyn4UTSfuebL6RS3-tFb3nWe9
!unzip merged_t.zip
!mkdir datasets/
!mv  merged_train4nlp_w_diff.parquet datasets/
!mv merged_test4nlp_w_diff.parquet datasets/
!mv merged_test.parquet datasets/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install evaluate
!pip install transformers[torch]
!pip install sentence_transformers
!pip install catboost

---

In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader, Dataset

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## Making test for CatBoost

In [ ]:
test = pd.read_parquet("datasets/merged_test4nlp_w_diff.parquet")
test

,variantid1,variantid2,full_text_1,full_text_2,text_difference
0,52076340,290590137,name: Батарейка AAA щелочная Perfeo LR03/10BL ...,name: Батарейка AAA щелочная Perfeo LR03/2BL m...,Товар 1 | name: Батарейка AAA щелочная Perfeo ...
1,64525522,204128919,"name: Смартфон Ulefone Armor X5 3/32 ГБ, черны...","name: Смартфон Ulefone Armor X3 2/32 ГБ, черны...",Товар 1 | name: Смартфон Ulefone Armor X5 3/32...
2,77243372,479860557,name: Цифровой кабель TV-COM HDMI 1.4 (M/ M) F...,name: Кабель HDMI 1.4 (Male/Male) (CG150S-1.5...,Товар 1 | name: Цифровой кабель TV-COM HDMI 1....
3,86065820,540678372,name: Игровая мышь проводная A4Tech Bloody P93...,"name: Мышь A4Tech Bloody P93s Bullet, серый, о...",Товар 1 | color: черный\nТовар 2 | color: серы...
4,91566575,258840506,"name: Смартфон Vivo Y93 1815 3/32 ГБ, черный\n...","name: Смартфон Vivo Y81 3/32 ГБ, черный\ncolor...","Товар 1 | color: черный, синий\nТовар 2 | colo..."
...,...,...,...,...,...
18079,666998614,667074522,name: Оперативная память Corsair VENGEANCE LPX...,name: Оперативная память Corsair VENGEANCE LPX...,Товар 1 | color: black\nТовар 1 | name: Операт...
18080,670036240,670048449,name: Зарядное устройство для HP 15-bw661ur бл...,name: Зарядное устройство для HP 15s-fq2066ur ...,Товар 1 | name: Зарядное устройство для HP 15-...
18081,670284509,684323809,name: Ремешок из натуральной кожи 20 мм для Sa...,name: Ремешок для часов UTHAI Z08 plus из нату...,"Товар 1 | color: желтый, красный\nТовар 2 | co..."
18082,692172005,704805270,name: Лазерный новогодний проектор для дома и ...,name: Новогодний проектор гирлянда морозостойк...,Товар 1 | color: синий\nТовар 2 | color: зелен...


### Quora Distilbert Duplicate Questions

In [ ]:
model = SentenceTransformer('distilbert-quora-full')

In [ ]:
model.eval()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
text1_embed = model.encode(test["full_text_1"], show_progress_bar=True)
text2_embed = model.encode(test["full_text_2"], show_progress_bar=True)

Batches:   0%|          | 0/566 [00:00<?, ?it/s]

Batches:   0%|          | 0/566 [00:00<?, ?it/s]

In [ ]:
cos_sim = np.array([cosine_similarity(
    [text1_embed[i]], [text2_embed[i]])[0][0] for i in range(len(text1_embed))])

cos_sim.max(), cos_sim.min(), cos_sim.mean()

(1.0000002, 0.046247303, 0.68829876)

In [ ]:
test["text_embed_cos_sim"] = cos_sim
test

,variantid1,variantid2,full_text_1,full_text_2,text_difference,text_embed_cos_sim
0,52076340,290590137,name: Батарейка AAA щелочная Perfeo LR03/10BL ...,name: Батарейка AAA щелочная Perfeo LR03/2BL m...,Товар 1 | name: Батарейка AAA щелочная Perfeo ...,0.311753
1,64525522,204128919,"name: Смартфон Ulefone Armor X5 3/32 ГБ, черны...","name: Смартфон Ulefone Armor X3 2/32 ГБ, черны...",Товар 1 | name: Смартфон Ulefone Armor X5 3/32...,0.768290
2,77243372,479860557,name: Цифровой кабель TV-COM HDMI 1.4 (M/ M) F...,name: Кабель HDMI 1.4 (Male/Male) (CG150S-1.5...,Товар 1 | name: Цифровой кабель TV-COM HDMI 1....,0.504581
3,86065820,540678372,name: Игровая мышь проводная A4Tech Bloody P93...,"name: Мышь A4Tech Bloody P93s Bullet, серый, о...",Товар 1 | color: черный\nТовар 2 | color: серы...,0.696828
4,91566575,258840506,"name: Смартфон Vivo Y93 1815 3/32 ГБ, черный\n...","name: Смартфон Vivo Y81 3/32 ГБ, черный\ncolor...","Товар 1 | color: черный, синий\nТовар 2 | colo...",0.815612
...,...,...,...,...,...,...
18079,666998614,667074522,name: Оперативная память Corsair VENGEANCE LPX...,name: Оперативная память Corsair VENGEANCE LPX...,Товар 1 | color: black\nТовар 1 | name: Операт...,0.372602
18080,670036240,670048449,name: Зарядное устройство для HP 15-bw661ur бл...,name: Зарядное устройство для HP 15s-fq2066ur ...,Товар 1 | name: Зарядное устройство для HP 15-...,0.602493
18081,670284509,684323809,name: Ремешок из натуральной кожи 20 мм для Sa...,name: Ремешок для часов UTHAI Z08 plus из нату...,"Товар 1 | color: желтый, красный\nТовар 2 | co...",0.404149
18082,692172005,704805270,name: Лазерный новогодний проектор для дома и ...,name: Новогодний проектор гирлянда морозостойк...,Товар 1 | color: синий\nТовар 2 | color: зелен...,0.939978


### ruBert Classification

In [ ]:
checkpoint = "rubert-base-class-full/checkpoint-34486"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
class TextTestDataset(Dataset):
    def __init__(self, data_df, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.sentences1 = data_df["full_text_1"].values
        self.sentences2 = data_df['full_text_2'].values

    def __len__(self):
        return self.sentences1.shape[0]

    def __getitem__(self, i):
        sentence1, sentence2 = self.sentences1[i], self.sentences2[i]

        tokens = tokenizer(sentence1, sentence2, truncation="longest_first", padding="max_length", max_length=self.max_length)

        tokens = {key: torch.tensor(val).long() for key, val in tokens.items()}

        return tokens


test_dataset = TextTestDataset(test, tokenizer)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=False)

In [ ]:
model = model.to("cuda")

In [ ]:
model.eval()

In [ ]:
all_preds = []

with torch.no_grad():
    for tokens in tqdm(test_dataloader):
        for key in tokens:
            tokens[key] = tokens[key].to("cuda")
        
        pred = model(**tokens)
        pred = pred['logits'].detach().cpu().numpy().tolist()

        all_preds.extend(pred)

  0%|          | 0/283 [00:00<?, ?it/s]

In [ ]:
test['rubert_pred_1'] = np.array([i for i in all_preds])[:,0]
test['rubert_pred_2'] = np.array([i for i in all_preds])[:,1]

test = test.drop(['full_text_1', 'full_text_2'], axis=1)
test

,variantid1,variantid2,text_difference,text_embed_cos_sim,rubert_pred_1,rubert_pred_2
0,52076340,290590137,Товар 1 | name: Батарейка AAA щелочная Perfeo ...,0.311753,1.539033,-1.460063
1,64525522,204128919,Товар 1 | name: Смартфон Ulefone Armor X5 3/32...,0.768290,1.513036,-1.391075
2,77243372,479860557,Товар 1 | name: Цифровой кабель TV-COM HDMI 1....,0.504581,1.077677,-0.684337
3,86065820,540678372,Товар 1 | color: черный\nТовар 2 | color: серы...,0.696828,0.160444,-0.120696
4,91566575,258840506,"Товар 1 | color: черный, синий\nТовар 2 | colo...",0.815612,1.067332,-0.660168
...,...,...,...,...,...,...
18079,666998614,667074522,Товар 1 | color: black\nТовар 1 | name: Операт...,0.372602,1.538683,-1.462968
18080,670036240,670048449,Товар 1 | name: Зарядное устройство для HP 15-...,0.602493,1.047127,-0.640164
18081,670284509,684323809,"Товар 1 | color: желтый, красный\nТовар 2 | co...",0.404149,0.625460,-0.347730
18082,692172005,704805270,Товар 1 | color: синий\nТовар 2 | color: зелен...,0.939978,-1.614517,1.508078


### XLM Roberta Regression

In [ ]:
checkpoint = "regression_model/checkpoint-34486"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
class TextTestDataset(Dataset):
    def __init__(self, data_df, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.sentences = data_df["text_difference"].values

    def __len__(self):
        return self.sentences.shape[0]

    def __getitem__(self, i):
        sentence = self.sentences[i]

        tokens = tokenizer(sentence, truncation="longest_first", padding="max_length", max_length=self.max_length)

        tokens = {key: torch.tensor(val) for key, val in tokens.items()}

        return tokens


test_dataset = TextTestDataset(test, tokenizer)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, drop_last=False)

In [ ]:
model = model.to("cuda")

In [ ]:
model.eval()

In [ ]:
all_preds = []

with torch.no_grad():
    for tokens in tqdm(test_dataloader):
        for key in tokens:
            tokens[key] = tokens[key].to("cuda")
        
        pred = model(**tokens)
        pred = pred['logits'].detach().cpu().numpy().tolist()

        all_preds.extend(pred)

  0%|          | 0/283 [00:00<?, ?it/s]

In [ ]:
test["xlm_roberta_pred"] = np.array(all_preds).flatten()

In [ ]:
test = test.drop('text_difference', axis=1)
test

,variantid1,variantid2,text_embed_cos_sim,rubert_pred_1,rubert_pred_2,xlm_roberta_pred
0,52076340,290590137,0.311753,1.539033,-1.460063,0.009385
1,64525522,204128919,0.768290,1.513036,-1.391075,0.198230
2,77243372,479860557,0.504581,1.077677,-0.684337,0.024874
3,86065820,540678372,0.696828,0.160444,-0.120696,0.793320
4,91566575,258840506,0.815612,1.067332,-0.660168,0.127452
...,...,...,...,...,...,...
18079,666998614,667074522,0.372602,1.538683,-1.462968,0.005983
18080,670036240,670048449,0.602493,1.047127,-0.640164,0.175674
18081,670284509,684323809,0.404149,0.625460,-0.347730,0.303050
18082,692172005,704805270,0.939978,-1.614517,1.508078,0.986750


In [ ]:
embs = pd.read_parquet("datasets/merged_test.parquet")

test = test.merge(embs, on=["variantid1", "variantid2"]).drop(['full_text_1', 'full_text_2'], axis=1)

In [ ]:
test["main_pic_embed_cos_sim"] = test.apply(lambda x: cosine_similarity([x['main_pic_embeddings_resnet_v1_1'][0]],
                                                                        [x['main_pic_embeddings_resnet_v1_2'][0]])[0][0], axis=1)

test = test.drop(['main_pic_embeddings_resnet_v1_1', 'main_pic_embeddings_resnet_v1_2'], axis=1)

test

,variantid1,variantid2,text_embed_cos_sim,rubert_pred_1,rubert_pred_2,xlm_roberta_pred,main_pic_embed_cos_sim
0,52076340,290590137,0.311753,1.539033,-1.460063,0.009385,0.997061
1,64525522,204128919,0.768290,1.513036,-1.391075,0.198230,0.776353
2,77243372,479860557,0.504581,1.077677,-0.684337,0.024874,0.994726
3,86065820,540678372,0.696828,0.160444,-0.120696,0.793320,0.881459
4,91566575,258840506,0.815612,1.067332,-0.660168,0.127452,0.787680
...,...,...,...,...,...,...,...
18079,666998614,667074522,0.372602,1.538683,-1.462968,0.005983,1.000000
18080,670036240,670048449,0.602493,1.047127,-0.640164,0.175674,0.946200
18081,670284509,684323809,0.404149,0.625460,-0.347730,0.303050,0.998958
18082,692172005,704805270,0.939978,-1.614517,1.508078,0.986750,1.000000


### Submit

In [ ]:
model = CatBoostClassifier()
model.load_model(
    'drive/MyDrive/models/catboost+xlm_roberta_regr_diff_2_eps+rubert_base_class_full_2_eps+distilbert_quora_full_6_eps+main_pics_cos_sim.cbm'
)

In [ ]:
submission = test.copy()
submission["target"] = model.predict_proba(test[test.columns[2:]])[:, 1]

submission = submission[["variantid1", "variantid2", "target"]]

In [ ]:
models_names = "xlm_roberta_regr_diff_2_eps+rubert_base_class_full_2_eps+distilbert_quora_full_6_eps+main_pics_cos_sim"

In [ ]:
save_path = f"submits/catboost+{models_names}.csv"

submission.to_csv(save_path, index=False)

pd.read_csv(save_path)

,variantid1,variantid2,target
0,52076340,290590137,0.036744
1,64525522,204128919,0.412169
2,77243372,479860557,0.053656
3,86065820,540678372,0.383063
4,91566575,258840506,0.421053
...,...,...,...
18079,666998614,667074522,0.036837
18080,670036240,670048449,0.129085
18081,670284509,684323809,0.091093
18082,692172005,704805270,0.913347


In [ ]:
print(save_path)